In [1]:
import pickle

# File path of the saved pickle
file_path = r"D:\combo_list.pickle"

# Load the data from the pickle file
with open(file_path, 'rb') as file:
    loaded_combo_list = pickle.load(file)

print("Combo list loaded successfully.")


Combo list loaded successfully.


In [2]:
import numpy as np
from PIL import Image
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

x_train = []
y_train = []

# Loop through each sub-array and read, resize, and normalize the images
label = 0
for sub_array in tqdm(loaded_combo_list):
    for image_path in sub_array:
        image = Image.open(image_path).convert("RGB")
        image = image.resize((32, 32))
        image = np.array(image) / 255.0  # Normalize to values between 0 and 1
        x_train.append(image)
        y_train.append(label)
    label += 1

x_train = np.array(x_train)
y_train = np.array(y_train)

# Convert y_train to categorical labels
y_train = to_categorical(y_train)

# Now, x_train contains the normalized and resized images, and y_train contains the categorical labels


In [5]:
np.shape(x_train)

(300000, 32, 32, 3)

In [7]:
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


# Load the pre-trained ResNet50 model (excluding the top layer)
base_model = ResNet50(weights=None, include_top=False, input_shape=(32, 32, 3))

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(512, activation='relu')(x)

# Add the output layer with the number of classes
output = Dense(y_train.shape[1], activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)


In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler

# Define the optimizer with SGD and momentum
optimizer = SGD(lr=0.001, momentum=0.9)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=100, validation_split=0.2)

3690/3750 [============================>.] - ETA: 3s - loss: 6.6717 - accuracy: 0.0040

In [ ]:
model.save('D:/1000class_model.h5')